# Pseudo Anonimização dos dados de Contribuinte do IPTU

In [1]:
import hashlib

In [2]:
hash_anonimizada = hashlib.md5('DADO PARA SER ANONIMIZADO'.encode('iso-8859-9')).hexdigest()

In [3]:
print(hash_anonimizada)

512141181c5031e9bcfcf7266c42a4b5


In [4]:
hash_procurada = hashlib.md5('DADO ANONIMO PARA SER PROCURADO'.encode('iso-8859-9')).hexdigest()

In [5]:
if hash_anonimizada == hash_procurada:
    print('Dados são iguais')
else:
    print('Os dados não conferem')

Os dados não conferem


## Carregando as bibliotecas

In [6]:
import pandas as pd

## Abrindo a base de IPTU

In [7]:
exercicio = '2020'

In [8]:
df_iptu = pd.read_csv(f'downloads/IPTU_{exercicio}.zip',\
          compression='zip',\
          encoding='iso-8859-9',\
          sep=';',\
          decimal=',',\
          error_bad_lines=False)

## Redefenindo colunas

In [9]:
colunas = list(df_iptu.columns)

In [10]:
colunas.remove('CPF/CNPJ DO CONTRIBUINTE 1')

In [11]:
colunas.remove('NOME DO CONTRIBUINTE 1')

In [12]:
colunas.remove('CPF/CNPJ DO CONTRIBUINTE 2')

In [13]:
colunas.remove('NOME DO CONTRIBUINTE 2')

In [14]:
colunas.insert(5,'HASH CONTRIBUINTE 1')

In [15]:
colunas.insert(7,'HASH CONTRIBUINTE 2')

In [16]:
colunas

['NUMERO DO CONTRIBUINTE',
 'ANO DO EXERCICIO',
 'NUMERO DA NL',
 'DATA DO CADASTRAMENTO',
 'TIPO DE CONTRIBUINTE 1',
 'HASH CONTRIBUINTE 1',
 'TIPO DE CONTRIBUINTE 2',
 'HASH CONTRIBUINTE 2',
 'NUMERO DO CONDOMINIO',
 'CODLOG DO IMOVEL',
 'NOME DE LOGRADOURO DO IMOVEL',
 'NUMERO DO IMOVEL',
 'COMPLEMENTO DO IMOVEL',
 'BAIRRO DO IMOVEL',
 'REFERENCIA DO IMOVEL',
 'CEP DO IMOVEL',
 'QUANTIDADE DE ESQUINAS/FRENTES',
 'FRACAO IDEAL',
 'AREA DO TERRENO',
 'AREA CONSTRUIDA',
 'AREA OCUPADA',
 'VALOR DO M2 DO TERRENO',
 'VALOR DO M2 DE CONSTRUCAO',
 'ANO DA CONSTRUCAO CORRIGIDO',
 'QUANTIDADE DE PAVIMENTOS',
 'TESTADA PARA CALCULO',
 'TIPO DE USO DO IMOVEL',
 'TIPO DE PADRAO DA CONSTRUCAO',
 'TIPO DE TERRENO',
 'FATOR DE OBSOLESCENCIA',
 'ANO DE INICIO DA VIDA DO CONTRIBUINTE',
 'MES DE INICIO DA VIDA DO CONTRIBUINTE',
 'FASE DO CONTRIBUINTE']

## Pseudo Anonimizando

É gerado uma [hash MD5 de 128 bits](https://pt.wikipedia.org/wiki/MD5) para cada coluna de CPF/CNPJ concatenada com o nome

In [17]:
c1 = df_iptu.loc[df_iptu['NOME DO CONTRIBUINTE 1'].notna()]

In [18]:
df_iptu['HASH CONTRIBUINTE 1'] = \
(c1['CPF/CNPJ DO CONTRIBUINTE 1'] + c1['NOME DO CONTRIBUINTE 1']).apply(
    lambda x: hashlib.md5(x.encode('iso-8859-9')).hexdigest()
)

In [19]:
c2 = df_iptu.loc[df_iptu['NOME DO CONTRIBUINTE 2'].notna()]

In [20]:
df_iptu['HASH CONTRIBUINTE 2'] = \
(c2['CPF/CNPJ DO CONTRIBUINTE 2'] + c2['NOME DO CONTRIBUINTE 2']).apply(
    lambda x: hashlib.md5(x.encode('iso-8859-9')).hexdigest()
)

## Salvando os dados processados Pseudo Anonimizados

In [21]:
df_iptu[colunas].to_csv(f'resultados/IPTU_{exercicio}_pseudoanonimizado.csv')